
# LCEL을 활용한 다중 프롬프트 & LLM 체인 연결 예시
## 1. LCEL의 정의와 배경

- **LCEL**은 **LangChain Expression Language**의 약자입니다.
- LangChain 0.1.17 이후 등장한 **최신 체인(Chain) 구축 방식**으로,
  프롬프트, LLM, 파서, 커스텀 함수 등 **모든 컴포넌트를 파이프라인(연산자 연결) 형태로 선언적 조합**할 수 있게 해줍니다.
- **목적:**  
  - 더 직관적이고, 유연하며, 유지보수/확장에 강한 LLM 워크플로우를  
    "마치 Unix 파이프라인이나 함수 합성"처럼 만들 수 있도록 설계

---

## 2. 기존 방식 vs LCEL 방식

- **전통적 LangChain (예: LLMChain)**
    - 객체 선언 및 체인 클래스 위주의 구조
    - 다단계 체인을 만들 때 번거로움이 있고, 코드 중복이 많아질 수 있음
    - 절차적으로 여러 단계를 나누어 처리해야 하며, 코드 가독성이 낮아질 수 있음

- **LCEL 방식 (`|` 파이프 연산자)**
    - 연산자(`|`) 기반의 선언적 파이프라인 구조
    - 단계 추가, 중간 변환, 조건 분기 등이 매우 쉬움
    - 한 줄의 직관적 코드로 복잡한 중첩 구조와 변환, 확장 처리가 자유롭게 가능
---

## 3. 핵심 문법 및 패턴

### 3.1. **기본 문법**

```python
chain = prompt | llm | StrOutputParser()
```python
chain = (
    first_prompt
    | llm
    | StrOutputParser()
    | {"title": lambda x: x}
    | second_prompt
    | llm
    | StrOutputParser()
)

## 1. 구조 및 목적

- **목적:**  
  하나의 파이프라인(chain)에서 여러 단계의 프롬프트-LLM 처리를 순차적으로 연결하여  
  입력값 → 1차 생성 → 중간 변환 → 2차 프롬프트 → 최종 생성  
  **다단계 생성 과정을 자동화**합니다.

---

## 2. 각 단계별 역할

1. **first_prompt**
    - 첫 번째 프롬프트 템플릿  
    - 예: `'입력값'에 대한 간단한 요약을 작성해줘.`

2. **llm**
    - LLM(Gemini, OpenAI 등) 모델 호출  
    - first_prompt에서 생성된 프롬프트에 따라 1차 응답 생성

3. **StrOutputParser()**
    - LLM 응답 객체(AIMessage 등)를 파이썬 문자열(str)로 변환

4. **{"title": lambda x: x}**
    - 딕셔너리 기반 필드 변환 단계  
    - 이전 결과(문자열)를 `title`이라는 이름의 필드로 매핑  
    - (입력값 형태를 `{ "title": 이전 결과 }`로 변환)  
    - 두 번째 프롬프트에서 `{title}` 변수로 활용 가능

5. **second_prompt**
    - 두 번째 프롬프트 템플릿  
    - 예: `'{title}'에 어울리는 해시태그를 추천해줘.`

6. **llm**
    - LLM으로 2차 생성  
    - 두 번째 프롬프트에 따라 최종 결과 생성

7. **StrOutputParser()**
    - 두 번째 LLM 응답도 파이썬 문자열로 변환

---

## 3. 전체 흐름 예시

- 입력값  
  → first_prompt  
  → LLM(1차 생성)  
  → 문자열 변환  
  → `{"title": ...}` 구조로 변환  
  → second_prompt  
  → LLM(2차 생성)  
  → 최종 문자열

---

## 4. 실무적 활용

- **다단계 생성 파이프라인:**  
  예: 초안 생성 → 요약 → 해시태그 추천, 질문 생성 → 답변 생성 → 평가 등
- 입력값 변환/가공 및 중간 데이터 재활용이 쉬움
- 각 단계의 결과를 명확하게 관리·확장 가능

---

## 5. 요약

- **LCEL(파이프라인 패턴)을 이용하면**  
  여러 프롬프트와 LLM을 직관적으로 연결하여  
  복잡한 생성형 워크플로우도 한 줄 코드로 간결하게 구현할 수 있습니다.

In [1]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
import os

# 환경변수 로드 및 API 키 확인
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError(".env 파일에 GOOGLE_API_KEY를 반드시 입력하세요.")

# Gemini LLM 객체 생성
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",    # 또는 "gemini-pro" 등 사용 가능
    google_api_key=api_key,
    temperature=0.7
)

# 첫 번째 체인: 아이디어 생성
first_prompt = PromptTemplate.from_template(
    "'{topic}'에 대한 블로그 글 제목을 하나 제안해주세요."
)

# 두 번째 체인: 개요 작성
second_prompt = PromptTemplate.from_template(
    "다음 제목의 블로그 글 개요를 작성해주세요: {title}"
)

# LCEL을 사용한 체인 연결
chain = (
    first_prompt
    | llm
    | StrOutputParser()
    | {"title": lambda x: x}
    | second_prompt
    | llm
    | StrOutputParser()
)

# 실행
result = chain.invoke({"topic": "인공지능과 미래"})
print(result)

## 인공지능과 미래: 판도라의 상자일까, 새로운 르네상스일까?  블로그 글 개요

**서론:**

* 인공지능(AI)의 급속한 발전과 그에 따른 사회적, 경제적 파장에 대한 소개.
* 본 글에서 다룰 핵심 주제: AI가 가져올 긍정적 미래와 부정적 미래 가능성의 양면적 고찰.  "판도라의 상자"와 "새로운 르네상스"라는 상반된 비유를 통해 독자의 흥미 유발.


**본론:**

**1. AI의 긍정적 미래: 새로운 르네상스의 가능성**

* **경제적 측면:** 생산성 향상, 새로운 산업 창출, 경제 성장 촉진 (구체적인 예시: 자동화, 개인 맞춤형 서비스, 의료 분야 발전 등)
* **사회적 측면:** 편리한 삶, 교육 및 의료 접근성 향상, 사회 문제 해결 (구체적인 예시: 스마트 도시, 자율주행 자동차, 질병 진단 및 치료 등)
* **과학기술적 측면:** 과학적 발견 가속화, 새로운 기술 혁신 (구체적인 예시: 우주 탐사, 기후변화 대응, 신소재 개발 등)
* 각 측면에 대한 구체적인 사례와 전문가 의견 제시.


**2. AI의 부정적 미래: 판도라의 상자의 위험성**

* **일자리 감소 및 사회적 불평등 심화:** 자동화에 따른 일자리 대체, 소득 불균형 심화 문제 제기.
* **AI 윤리 문제:** 알고리즘 편향, 프라이버시 침해, AI 무기 개발 등 윤리적 문제점 분석.
* **AI의 통제 불가능성:**  AI의 자율성 증가에 따른 예측 불가능한 결과와 위험성 제기. (예: 슈퍼지능의 등장, AI의 악용 가능성 등)
* 각 측면에 대한 구체적인 사례와 전문가 의견 제시.  미래 사회의 잠재적 위험에 대한 심도있는 고찰.


**결론:**

* AI의 양면성을 재차 강조하며, 낙관적 전망과 비관적 전망 모두에 대한 균형 잡힌 시각 제시.
* AI의 잠재력을 긍정적으로 활용하기 위한 방안 모색:  윤리적 가이드라인 제정, 교육 및 사회 시스템 개혁, 국제적 협력 강화 등.
* AI 기술 발전에 대한 지속적인 관심과 책임감 있는 접근의 중요성 강조